In [1]:
import sys
sys.path.append('../../')
from utils import print_check_gpt_results

In [2]:
NUM_FILES = 350
FOLDER_NAME = '../random_samples_formatting'
GPT_SAVED_FILE_NAME = 'formatted_code_gpt'
GPT_SAVED_FOLDER_NAME = 'reformatted_gpt'

In [3]:
# read in files from folder random_cells
random_cells = []

for i in range(NUM_FILES):
    file_name = f'{FOLDER_NAME}/{i}.py'
    with open(file_name, 'r') as f:
        random_cells.append(f.read())

In [5]:
task = """Format the code delimited by triple backticks according to PEP 8 conventions. Do not add, remove, or change anything else. Structure your response under the following headings: 'Identified formatting issues' (a short paragraph describing the formatting issues) and 'Formatted code' (the full code with all formatting issues fixed)."""

shots_input = []
shots_output = []
for i in range(1, 5):
    with open(f'../shots/shot{i}.py', 'r') as f:
        shots_input.append(f.read())
    with open(f'../shots/shot{i}_after.py', 'r') as f:
        shots_output.append(f.read())

ex1_input = f"""```python
{shots_input[0]}
```"""

ex1_output = f"""Identified formatting issues:
- Imports 'utils' and 'rain' should not be on the same line
- The indentation in the function 'evaluate' contains tabs
- The function 'evaluate' should have two tabs after its definition, not one

Formatted code:
```python
{shots_output[0]}
```"""

ex2_input = f"""```python
{shots_input[1]}
```"""

ex2_output = f"""Identified formatting issues:
- Arithmetic operators should have whitespace around them
- There is a line break after the binary operator but the line break should be before the binary operator
- No statements should end with a semicolon

Formatted code:
```python
{shots_output[1]}
```"""

ex3_input = f"""```python
{shots_input[2]}
```"""

ex3_output = f"""Identified formatting issues:
- Blank line contains whitespace but it should not
- The if statement is too long and should be split into multiple lines
- The code block inside the if-statement is not indented with 4 spaces

Formatted code:
```python
{shots_output[2]}
```"""

ex4_input = f"""```python
{shots_input[3]}
```"""

ex4_output = f"""Identified formatting issues:
- There should be two blank lines between the imports and the first function not one
- The return statement in the function 'find' is too long and should be split into multiple lines
- No blank lines should contain whitespace
- The continuation line for the print statement is under-indented
- The closing bracket of the variable assignment 'updated_offset' does not match the indentation of the opening bracket's line

Formatted code:
```python
{shots_output[3]}
```"""

In [6]:
# Estimate cost
import sys
sys.path.append("../../..")
import utils

def estimate_tokens():
    in_tok = ''
    out_tok = ''
    for i, cell_src in enumerate(random_cells):
        # estimate prompt
        in_tok += task + ex1_input + ex1_output + ex2_input + ex2_output + ex3_input + ex3_output + ex4_input + ex4_output
        in_tok += f"```python\n{cell_src}\n```"
        # estimate response
        out_tok += cell_src
    return in_tok, out_tok

in_tok, out_tok = estimate_tokens()

utils.gpt_35_turbo_token_dollar_cost(in_tok, out_tok)

0.5412015

In [7]:
# # identify unused functions using vulture (GPT)
# import openai
# openai.api_key = my_key

# # GPT
# def format_code(cell_src):
#     while True:
#         try:
#             completion = openai.ChatCompletion.create(
#                 model="gpt-3.5-turbo",
#                 temperature=0,
#                 messages = [
#                 {"role": "user", "content": task},
#                 {"role": "user", "content": ex1_input},
#                 {"role": "assistant", "content": ex1_output},
#                 {"role": "user", "content": ex2_input},
#                 {"role": "assistant", "content": ex2_output},
#                 {"role": "user", "content": ex3_input},
#                 {"role": "assistant", "content": ex3_output},
#                 {"role": "user", "content": ex4_input},
#                 {"role": "assistant", "content": ex4_output},
#                 {"role": "user", "content": f"```python\n{cell_src}\n```"},
#             ]
#             )
#         except Exception as e:
#             if 'maximum context length' in str(e):
#                 print('...Error.. too long...' + str(e))
#                 return 'length', None
#             else:
#                 print('...Error.. trying again...' + str(e))
#         else:
#             break
#     return completion.choices[0].finish_reason, completion.choices[0].message["content"]

# gpt_results = []
# for i, cell_src in enumerate(random_cells):
#     print(f'Processing file {i}')
#     finish_reason, result = format_code(cell_src)
#     print(f'File {i} - {finish_reason}')
#     gpt_results.append({'reason': finish_reason, 'result': result})

# # save the results to a file
# with open(GPT_SAVED_FILE_NAME, 'w') as f:
#     f.write(str(gpt_results))

In [8]:
# read in gpt result from file
with open(GPT_SAVED_FILE_NAME, 'r') as f:
    gpt_results = eval(f.read())

In [9]:
sys.path.append('../')
from common import pycodestyle, group_by_error, print_num_reductions, print_percentage_difference, unpack_gpt, IGNORE_TYPES

In [10]:
gpt_formatted_code, gpt_changes = unpack_gpt(gpt_results)

In [11]:
# print all to new folder
import os
if not os.path.exists(GPT_SAVED_FOLDER_NAME):
    os.makedirs(GPT_SAVED_FOLDER_NAME)
for i, code in enumerate(gpt_formatted_code):
    with open(f'{GPT_SAVED_FOLDER_NAME}/{i}.py', 'w') as f:
        if gpt_formatted_code[i] is None or gpt_changes[i] is None:
            f.write(random_cells[i])
        else:
            f.write(code)

In [12]:
# store error counts in a hash
error_counts_before = pycodestyle(FOLDER_NAME, NUM_FILES, IGNORE_TYPES)

# print the error counts
total_errors_before = sum(error_counts_before.values())
print(f'Total before: {total_errors_before}')

Total before: 3062


In [13]:
# store error counts in a hash
error_counts_after = pycodestyle(GPT_SAVED_FOLDER_NAME, NUM_FILES, IGNORE_TYPES)

# print the error counts
total_errors_after = sum(error_counts_after.values())
print(f'Total after: {total_errors_after}')

Total after: 944


In [14]:
error_counts_before

{'E501': 447,
 'E226': 181,
 'E231': 592,
 'E266': 80,
 'E303': 8,
 'W293': 213,
 'E251': 359,
 'E265': 130,
 'W291': 142,
 'E202': 29,
 'E203': 47,
 'E241': 16,
 'E225': 154,
 'E117': 2,
 'E128': 33,
 'E302': 48,
 'E401': 2,
 'E703': 52,
 'E221': 76,
 'E305': 27,
 'E228': 5,
 'E402': 32,
 'E111': 141,
 'E114': 22,
 'E261': 46,
 'E722': 2,
 'E201': 29,
 'E275': 9,
 'E126': 4,
 'E127': 3,
 'E211': 13,
 'E271': 2,
 'E262': 27,
 'E113': 2,
 'E116': 8,
 'W504': 6,
 'E123': 2,
 'E101': 1,
 'W191': 2,
 'E701': 3,
 'E121': 33,
 'E131': 3,
 'E702': 9,
 'E741': 2,
 'E115': 3,
 'E124': 1,
 'E222': 8,
 'E301': 2,
 'E712': 1,
 'E122': 3}

In [15]:
error_counts_after

{'E501': 456,
 'E226': 46,
 'W293': 116,
 'E225': 19,
 'E231': 34,
 'E266': 38,
 'W291': 49,
 'E302': 12,
 'E305': 5,
 'E228': 3,
 'E722': 1,
 'E113': 6,
 'E275': 5,
 'E265': 19,
 'E402': 6,
 'W504': 11,
 'E111': 13,
 'E116': 2,
 'E741': 1,
 'E211': 4,
 'E221': 26,
 'E241': 7,
 'E703': 7,
 'E127': 1,
 'E128': 1,
 'E122': 1,
 'E222': 1,
 'E301': 1,
 'E712': 1,
 'E261': 1,
 'E262': 1,
 'E251': 48,
 'E115': 2}

In [16]:
# List percentage difference between before and after for total
print(f'Total percentage difference: {(total_errors_after - total_errors_before) / total_errors_before * 100}%')

Total percentage difference: -69.17047681254083%


In [17]:
error_counts_grouped_before = group_by_error(error_counts_before)
error_counts_grouped_before

{'E1': 261,
 'E2': 1803,
 'E3': 85,
 'E4': 34,
 'E5': 447,
 'E7': 69,
 'E9': 0,
 'W1': 2,
 'W2': 355,
 'W3': 0,
 'W5': 6,
 'W6': 0}

In [18]:
error_counts_grouped_after = group_by_error(error_counts_after)
error_counts_grouped_after

{'E1': 26,
 'E2': 252,
 'E3': 18,
 'E4': 6,
 'E5': 456,
 'E7': 10,
 'E9': 0,
 'W1': 0,
 'W2': 165,
 'W3': 0,
 'W5': 11,
 'W6': 0}

In [19]:
print_num_reductions(error_counts_grouped_before, error_counts_grouped_after)

('E1', 'Indentation'): 261 -> 26
('E2', 'Whitespace'): 1803 -> 252
('E3', 'Blank line'): 85 -> 18
('E4', 'Import'): 34 -> 6
('E5', 'Line length'): 447 -> 456
('E7', 'Statement'): 69 -> 10
('E9', 'Runtime'): 0 -> 0
('W1', 'Indentation warning'): 2 -> 0
('W2', 'Whitespace warning'): 355 -> 165
('W3', 'Blank line warning'): 0 -> 0
('W5', 'Line break warning'): 6 -> 11
('W6', 'Deprecation warning'): 0 -> 0


In [20]:
print_percentage_difference(error_counts_grouped_before, error_counts_grouped_after)

('E1', 'Indentation'): -90.04%
('E2', 'Whitespace'): -86.02%
('E3', 'Blank line'): -78.82%
('E4', 'Import'): -82.35%
('E5', 'Line length'): 2.01%
('E7', 'Statement'): -85.51%
('E9', 'Runtime'): Undefined
('W1', 'Indentation warning'): -100.0%
('W2', 'Whitespace warning'): -53.52%
('W3', 'Blank line warning'): Undefined
('W5', 'Line break warning'): 83.33%
('W6', 'Deprecation warning'): Undefined
